In [1]:
import requests
from collections import OrderedDict
import logging as lg
import pandas as pd

import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Template to request place information from nominatim API for OSM

In [2]:
url_template = 'https://nominatim.openstreetmap.org/search?format=json&limit=1&q={}'

Build a query to make request

In [3]:
query = 'London, Greater London, England, United Kingdom'

In [4]:
url = url_template.format(query)

Make a request and save JSON

In [5]:
response = requests.get(url, timeout=60)
results = response.json()
results

[{'boundingbox': ['51.2867602', '51.6918741', '-0.510375', '0.3340155'],
  'class': 'place',
  'display_name': 'London, Greater London, England, UK',
  'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_place_city.p.20.png',
  'importance': 1.2954895765402,
  'lat': '51.5073219',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
  'lon': '-0.1276473',
  'osm_id': '65606',
  'osm_type': 'relation',
  'place_id': '207215760',
  'type': 'city'}]

Get lat and lon from the results

In [6]:
if len(results) > 0 and 'lat' in results[0] and 'lon' in results[0]:
    lat= float(results[0]['lat'])
    lon=float(results[0]['lon'])
    point= (lat, lon)
    print('Geocoded "{}" to {}'.format(query, point))
else:
    raise Exception('Nominatim geocoder returned no results for query "{}"'.format(query))

Geocoded "London, Greater London, England, United Kingdom" to (51.5073219, -0.1276473)


In [7]:
#get bounding box 
bbox_south, bbox_north, bbox_west, bbox_east = [float(x) for x in results[0]['boundingbox']]
bbox_south, bbox_west,bbox_north, bbox_east

(51.2867602, -0.510375, 51.6918741, 0.3340155)

Connecto to Overpass API

In [8]:
#send a request to the Overpass API via HTTP POST 
#and return the JSON response. 

#set HTTP url call
url_template='http://overpass-api.de/api/interpreter?data={};{};out;'
output = '[out:json]'

In [9]:
#this query is built by especifying a node and a bounding box
bbox="({},{},{},{})"
bbox=bbox.format(bbox_south, bbox_west,bbox_north, bbox_east)
query = 'node["amenity"]{}'
query = query.format(bbox)
url = url_template.format(output, query)

#make request (nodes)
response = requests.get(url, timeout=60)
json_dict_data = response.json()

json_dict_data #this is a dictionary

{'elements': [{'id': 108042,
   'lat': 51.5235442,
   'lon': -0.1355991,
   'tags': {'addr:housenumber': '31',
    'addr:street': 'University Street',
    'amenity': 'pub',
    'name': 'Simmons Bar',
    'old_name': 'The Jeremy Bentham',
    'opening_hours': 'Mo-We 15:00-23:30; Th,Fr 12:00-24:00; Su 15:00-23:30',
    'phone': '020 7387 3033',
    'postal_code': 'WC1E 6JL',
    'randomjunk_bot': 'reverted edit by FilipeOliveira on 2008-11-24T15:02:32+00:00',
    'toilets': 'yes',
    'toilets:access': 'customers',
    'toilets:wheelchair': 'no',
    'website': 'http://www.simmonsbar.co.uk/euston-square/4593769006',
    'wheelchair': 'limited'},
   'type': 'node'},
  {'id': 108539,
   'lat': 51.5291251,
   'lon': -0.0933878,
   'tags': {'amenity': 'bicycle_rental',
    'capacity': '24',
    'name': 'Windsor Terrace',
    'network': 'tfl_cycle_hire'},
   'type': 'node'},
  {'id': 109575,
   'lat': 51.5282624,
   'lon': -0.1431214,
   'tags': {'amenity': 'advice', 'name': 'Citizens advice 

In [10]:
#this query is built by especifying a way(nodes delimitating an amenity) and a bounding box
bbox="({},{},{},{})"
bbox=bbox.format(bbox_south, bbox_west,bbox_north, bbox_east)
query2 = 'way["amenity"]{}'
query2 = query2.format(bbox)
url2 = url_template.format(output, query2)

#make request (ways)
response2 = requests.get(url2, timeout=60)
json_dict_data2 = response2.json()

json_dict_data2 #this is a dictionary

{'elements': [{'id': 28303,
   'nodes': [727341145,
    727341199,
    727341143,
    727341142,
    316177975,
    635021,
    809024,
    809025,
    809027,
    20927803,
    809028,
    19828968,
    809029,
    19828955,
    316177976,
    19828954,
    727341145],
   'tags': {'access': 'customers',
    'amenity': 'parking',
    'operator': 'Homebase',
    'parking': 'surface'},
   'type': 'way'},
  {'id': 75215,
   'nodes': [1598418417, 649321, 649322, 1598418419, 1598418417],
   'tags': {'amenity': 'parking'},
   'type': 'way'},
  {'id': 1080949,
   'nodes': [640171, 640174, 640175, 1739214039, 4524995, 640173, 640171],
   'tags': {'access': 'permissive',
    'amenity': 'parking',
    'parking': 'surface'},
   'type': 'way'},
  {'id': 2956081,
   'nodes': [648041,
    648042,
    648043,
    21596295,
    648044,
    648045,
    648039,
    648046,
    648047,
    648048,
    648040,
    648041],
   'tags': {'amenity': 'parking', 'created_by': 'JOSM'},
   'type': 'way'},
  {'id'

    Export dictionary to a list

In [11]:
#nodes list
json_list_data = json_dict_data['elements']
json_list_data



[{'id': 108042,
  'lat': 51.5235442,
  'lon': -0.1355991,
  'tags': {'addr:housenumber': '31',
   'addr:street': 'University Street',
   'amenity': 'pub',
   'name': 'Simmons Bar',
   'old_name': 'The Jeremy Bentham',
   'opening_hours': 'Mo-We 15:00-23:30; Th,Fr 12:00-24:00; Su 15:00-23:30',
   'phone': '020 7387 3033',
   'postal_code': 'WC1E 6JL',
   'randomjunk_bot': 'reverted edit by FilipeOliveira on 2008-11-24T15:02:32+00:00',
   'toilets': 'yes',
   'toilets:access': 'customers',
   'toilets:wheelchair': 'no',
   'website': 'http://www.simmonsbar.co.uk/euston-square/4593769006',
   'wheelchair': 'limited'},
  'type': 'node'},
 {'id': 108539,
  'lat': 51.5291251,
  'lon': -0.0933878,
  'tags': {'amenity': 'bicycle_rental',
   'capacity': '24',
   'name': 'Windsor Terrace',
   'network': 'tfl_cycle_hire'},
  'type': 'node'},
 {'id': 109575,
  'lat': 51.5282624,
  'lon': -0.1431214,
  'tags': {'amenity': 'advice', 'name': 'Citizens advice bureau'},
  'type': 'node'},
 {'id': 11007

In [12]:
#ways list
json_list_data2 = json_dict_data2['elements']
json_list_data2

[{'id': 28303,
  'nodes': [727341145,
   727341199,
   727341143,
   727341142,
   316177975,
   635021,
   809024,
   809025,
   809027,
   20927803,
   809028,
   19828968,
   809029,
   19828955,
   316177976,
   19828954,
   727341145],
  'tags': {'access': 'customers',
   'amenity': 'parking',
   'operator': 'Homebase',
   'parking': 'surface'},
  'type': 'way'},
 {'id': 75215,
  'nodes': [1598418417, 649321, 649322, 1598418419, 1598418417],
  'tags': {'amenity': 'parking'},
  'type': 'way'},
 {'id': 1080949,
  'nodes': [640171, 640174, 640175, 1739214039, 4524995, 640173, 640171],
  'tags': {'access': 'permissive', 'amenity': 'parking', 'parking': 'surface'},
  'type': 'way'},
 {'id': 2956081,
  'nodes': [648041,
   648042,
   648043,
   21596295,
   648044,
   648045,
   648039,
   648046,
   648047,
   648048,
   648040,
   648041],
  'tags': {'amenity': 'parking', 'created_by': 'JOSM'},
  'type': 'way'},
 {'id': 2956183,
  'nodes': [629135, 629136, 629132, 629133, 14464652, 62

In [21]:
# column title for nodes dataframe

labels = ['id', 'lat', 'lon', 'amenity' ]
df = pd.DataFrame.from_records(json_list_data, columns = labels)
df



,id,lat,lon,amenity
0,108042,51.523544,-0.135599,NaN
1,108539,51.529125,-0.093388,NaN
2,109575,51.528262,-0.143121,NaN
3,110075,51.534257,-0.140276,NaN
4,451152,51.600840,-0.194608,NaN
5,451153,51.602031,-0.193503,NaN
6,451154,51.599586,-0.196005,NaN
7,451156,51.599632,-0.197601,NaN
8,451271,51.614104,-0.176505,NaN
9,469648,51.632361,-0.200258,NaN


In [ ]:
length_list = len(json_list_data)

for i in range(length_list):
    new = json_list_data[i]['tags']['amenity']
    df['amenity'][i] = new

/Users/mercedeslanda/Applications/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
df

In [19]:
#column title for ways dataframe

labels2 = ['id', 'type', 'first node', 'amenity' ]
df2 = pd.DataFrame.from_records(json_list_data2, columns = labels2)
df2

,id,type,first node,amenity
0,28303,way,NaN,NaN
1,75215,way,NaN,NaN
2,1080949,way,NaN,NaN
3,2956081,way,NaN,NaN
4,2956183,way,NaN,NaN
5,2956184,way,NaN,NaN
6,2956185,way,NaN,NaN
7,2956187,way,NaN,NaN
8,2956473,way,NaN,NaN
9,3042477,way,NaN,NaN


In [ ]:
length_list2 = len(json_list_data2)

for i in range(length_list2):
    new = json_list_data2[i]['tags']['amenity']
    df['amenity'][i] = new

In [20]:
length_list2 = len(json_list_data2)

for i in range(length_list2):
    new = json_list_data2[i]['nodes'][1]
    df['first node'][i] = new

KeyError: 'first node'